In [266]:
import pandas as pd
from rapidfuzz import fuzz
from itertools import combinations
from collections import defaultdict
import json

In [267]:
df2016=pd.read_excel('kenya-accidents-database.xlsx',sheet_name='2016')
df2017=pd.read_excel('kenya-accidents-database.xlsx',sheet_name='2017')

In [268]:
df2016.head()

,TIME 24 HOURS,BASE/SUB BASE,COUNTY,ROAD,PLACE,BRIEF ACCIDENT DETAILS,GENDER,AGE,CAUSE CODE,VICTIM,NO.,Date DD/MM/YYYY,Unnamed: 12
0,630,KITUI,MAKUENI,KITUI-ITHOKWE,KITUI SCHOOL,HEAD ON COLLISION,M,26,7.0,M/CYCLIST,1,2016-06-25,so MM/DD/YYYY is the solution :)
1,830,VOI,TAITA TAVETA,MOMBASA-NAIROBI,IKANGA,HEAD ON COLLISION,M,28,25.0,M/CYCLIST,1,2016-06-25,NaN
2,1330,MARIAKANI,KILIFI,MOMBASA-NAIROBI,KATOLANI,THE UNKNOWN M/V HIT THE MOTOR CYCLE,M,A & J,98.0,M/CYCLIST,1,2016-06-25,NaN
3,2100,ONGATA RONGAI,NAKURU,NAKURU-NAIROBI,MAASAI LODGE,THE VEHICLE KNOCKED DOWN A PEDESTRIAN WHO WAS ...,M,65,29.0,PEDESTRIAN,1,2016-06-25,NaN
4,1900,MATUU,MACHAKOS,MATUU-MWINGI,KIVANDINI,THE VEHICLE OVERTOOK A M/CYCLE AND LOST CONTRO...,M,A,10.0,PASSENGER,1,2016-06-25,NaN


In [269]:
for x in df2016['ROAD']:
    
    if isinstance(x, str) and '-' in x:
        x = x.replace('-', ' ')

df2016['ROAD'].unique() 

array(['KITUI-ITHOKWE', 'MOMBASA-NAIROBI', 'NAKURU-NAIROBI',
       'MATUU-MWINGI', 'KATITO-KENDU BAY', 'MAKUTANO-MWEA',
       'KENYATTA AVENUE', 'EASTERN BY PASS', 'NYERI-THIKA',
       'LIKONI-LINGALUNGA', 'KENDU BAY-KASITI', 'MERU-MAUA',
       'NAKURU NAIROBI', 'MIGORI KISII', 'KEROKA KISII',
       'NJORO MAU NAROK', 'HAILE SELASIE ROAD', 'GITHUNGURI ROAD',
       'MOMBASA NAIROBI', 'HAILE SELASIE', 'BONDO USENGE', 'RUIRU KIAMBU',
       'KIMILILI CHWELE ROAD', 'NGONG NAIROBI', 'KILIFI MSA ROAD',
       'THIKA SUPER HIGHWAY', 'MSAMBWENI UKUNDA', 'MERU MAUA',
       'NAIROBI MOMBASA', 'ELDORET KITALE', 'LANDHIES', 'KAMOROCK',
       'LUANDA KOTIENO NDORI', 'SOUTHERN BY PASS', 'KITENGELA ISINYA',
       'NAIVASHA NAIROBI', 'MOMBASA', 'KILIFI-MALINDI', 'NAKURU-ELDORET',
       'JOGOO', 'MSA VOI ROAD', 'KILIFI MOMBASA', 'KIJABE GICHIEGO ROAD',
       'PARKLANDS MUTHAMA', 'KISII KILGORIS ROAD', 'KISII MIGORI',
       'KISERIAN ISINYA', 'KITALE WEBUYE', 'ELDORET NAKURU',
       'NAROK 

In [270]:
df2016['COUNTY'].value_counts()

COUNTY
NAIROBI            71
NAKURU             32
KIAMBU             31
MAKUENI            26
KILIFI             18
HOMA BAY           14
MOMBASA            13
KAJIADO            12
KISII              12
KAKAMEGA           10
BUNGOMA            10
EMBU               10
NAROK               9
MACHAKOS            9
NYERI               8
UASIN GISHU         7
KISUMU              7
BARINGO             6
MERU                5
TRANS NZOIA         5
MIGORI              5
KERICHO             5
TAITA TAVETA        4
BOMET               4
SIAYA               4
VIHIGA              4
KWALE               4
KIRINYAGA           3
GARISSA             3
MURANG'A            3
BUSIA               3
MWINGI              3
MURANGA             3
LAIKIPIA            2
ELGEYO MARAKWET     2
KITUI               2
NANDI               1
KERCHO              1
NYAMIRA             1
MAKURU              1
NYAHURURU           1
MARAKWET            1
NYANDARUA           1
Name: count, dtype: int64

In [271]:
df2016.groupby('COUNTY')['ROAD'].value_counts()


COUNTY       ROAD              
BARINGO      ELDAMA RAVINE SAOS    1
             KABARNET MARIGAT      1
             KITALE ELDORET        1
             MARIGAT LORUK         1
             MUNGOTIO RAVINE       1
                                  ..
UASIN GISHU  KERICHO-KISUMU        1
             KITALE WEBUYE         1
VIHIGA       CHAVAKALI KAPSABET    2
             KAKAMEGA KISUMU       1
             KISUMU BUSIA          1
Name: count, Length: 283, dtype: int64

In [272]:
def roads_in_county(county_name):
    return list(df2016[df2016['COUNTY'] == county_name]['ROAD'].unique())
counties=[x for x in df2016['COUNTY'].unique() if isinstance(x, str)]
num_roads_per_county={county: len(roads_in_county(county)) for county in counties}
print(num_roads_per_county)



{'MAKUENI': 10, 'TAITA TAVETA': 3, 'KILIFI': 12, 'NAKURU': 22, 'MACHAKOS': 7, 'HOMA BAY': 13, 'KIRINYAGA': 3, 'NAIROBI': 45, 'KIAMBU': 24, 'MOMBASA': 13, 'MERU': 4, 'MIGORI': 3, 'KISII': 9, 'SIAYA': 4, 'BUNGOMA': 8, 'KWALE': 4, 'NANDI': 1, 'TRANS NZOIA': 3, 'KAJIADO': 11, 'UASIN GISHU': 5, 'NAROK': 8, 'KERCHO': 1, 'KAKAMEGA': 9, 'NYERI': 7, 'KERICHO': 4, 'EMBU': 8, 'KISUMU': 7, 'MURANGA': 3, 'BUSIA': 2, "MURANG'A": 3, 'VIHIGA': 3, 'GARISSA': 2, 'BARINGO': 6, 'LAIKIPIA': 2, 'MWINGI': 2, 'BOMET': 4, 'NYAMIRA': 1, 'ELGEYO MARAKWET': 2, 'MAKURU': 1, 'NYAHURURU': 1, 'KITUI': 1, 'MARAKWET': 1, 'NYANDARUA': 1}


In [273]:

county_road={}
for county in counties:
    
    county_road[county] = roads_in_county(county)

print(county_road)


{'MAKUENI': ['KITUI-ITHOKWE', 'NAIROBI MOMBASA', 'MOMBASA-NAIROBI', 'KANGUNDO NAIROBI', 'WOTE MAKINDU', 'KABATI TOWNSHIP', 'MUSYINI KYULU MURRAM ROAD', 'MOMBASA NAIROBI', 'KIKOKO NUNGUNI', 'MTITO TOWNSHIP'], 'TAITA TAVETA': ['MOMBASA-NAIROBI', 'NAIROBI MOMBASA', 'MWATATE VOI'], 'KILIFI': ['MOMBASA-NAIROBI', 'MOMBASA NAIROBI', 'KILIFI MSA ROAD', 'KILIFI-MALINDI', 'KILIFI MOMBASA', 'MARIAKANI-KALOLENI', 'MALINDI MALANGA', 'KILIFI VITENGENI', 'MALINDI MOMBASA', 'NAIROBI MOMBASA', 'KILIFI MALINDI', 'MOMBASA KILIFI'], 'NAKURU': ['NAKURU-NAIROBI', 'KENYATTA AVENUE', 'NAKURU NAIROBI', 'NJORO MAU NAROK', 'NAKURU-ELDORET', 'ELDORET NAKURU', 'ELEMENTAITA-GACHUMA', 'NAKURU GILGIL', 'LANET NDUNDURI', 'NAROK MAIMAHIU', 'OLENGURUONE ROAD', 'NAKURU-ELEMENTAITA', 'NJORO-ELBURGON', 'NAIROBI NAKURU', 'GILGIL NAKURU', 'OLKALAU GILGIL', 'MAI MAHIU NAIVASHA', 'NAKURU ELDORET', 'KERICHO NAKURU', 'MOLO ELBAGON', 'KENYATTA', 'MOGOTIO NAKURU'], 'MACHAKOS': ['MATUU-MWINGI', 'MACHAKOS KITUI RD', 'MATUU-THIKA', '

In [274]:
import re

def normalize_road(road_name):
    # Step 1: upper case and strip
    road_name = road_name.strip().upper()
    # Step 2: normalize separators
    road_name = re.sub(r'[-_]+', ' ', road_name)
    road_name = re.sub(r'\s+', ' ', road_name)
    # Step 3: normalize order for two-part names
    parts = road_name.split()
    if len(parts) == 2:
        parts = sorted(parts, reverse=True)
        road_name = ' '.join(parts)
    if road_name.endswith(' ROAD') or road_name.endswith(' RD') :
        road_name = road_name.rsplit(' ', 1)[0]
    return road_name

df2016['NORMALIZED_ROAD'] = df2016['ROAD'].apply(lambda x: normalize_road(x) if isinstance(x, str) else x)
df2016[['ROAD', 'NORMALIZED_ROAD']].head(10)

,ROAD,NORMALIZED_ROAD
0,KITUI-ITHOKWE,KITUI ITHOKWE
1,MOMBASA-NAIROBI,NAIROBI MOMBASA
2,MOMBASA-NAIROBI,NAIROBI MOMBASA
3,NAKURU-NAIROBI,NAKURU NAIROBI
4,MATUU-MWINGI,MWINGI MATUU
5,KATITO-KENDU BAY,KATITO KENDU BAY
6,MAKUTANO-MWEA,MWEA MAKUTANO
7,KENYATTA AVENUE,KENYATTA AVENUE
8,EASTERN BY PASS,EASTERN BY PASS
9,NYERI-THIKA,THIKA NYERI


In [275]:
print(len(df2016['ROAD'].unique()))
print(len(df2016['NORMALIZED_ROAD'].unique()))

258
218


In [276]:
import difflib

roads = df2016['NORMALIZED_ROAD'].unique()

threshold = 0.8  # similarity from 0 to 1

similar_pairs = []

for i, road1 in enumerate(roads):
    for road2 in roads[i+1:]:
        ratio = difflib.SequenceMatcher(None, road1, road2).ratio()
        if ratio >= threshold:
            similar_pairs.append((road1, road2, ratio))

for r1, r2, score in similar_pairs:
    print(f"{r1} <--> {r2} | Similarity: {score:.2f}")

KATITO KENDU BAY <--> KATITU KENDU BAY | Similarity: 0.94
EASTERN BY PASS <--> SOUTHERN BY PASS | Similarity: 0.84
KENDU BAY KASITI <--> KENDU BAY KATITO | Similarity: 0.88
KENDU BAY KASITI <--> KENDU BAY KATITU | Similarity: 0.88
THIKA SUPER HIGHWAY <--> THIKA SUPERHIGHWAY | Similarity: 0.97
THIKA SUPER HIGHWAY <--> THIUKA SUPER HIGHWAY | Similarity: 0.97
SOUTHERN BY PASS <--> NORTHERN BY PASS | Similarity: 0.88
NAIVASHA NAIROBI <--> NAIVASHA NAIROBI HIGHWAY | Similarity: 0.80
NAKURU ELDORET <--> NYARU ELDORET | Similarity: 0.89
KENDU BAY KATITO <--> KENDU KATITO | Similarity: 0.86
KENDU BAY KATITO <--> KENDU BAY KATITU | Similarity: 0.94
THIKA GATHIMA <--> THIKA GATURA | Similarity: 0.80
THIKA SUPERHIGHWAY <--> THIUKA SUPER HIGHWAY | Similarity: 0.95
THIKA GARISSA <--> THIKA GATURA | Similarity: 0.80
NAIROBI KIKUYU <--> NAIROBI KISUMU | Similarity: 0.86
OYUGIS KISII <--> OYUGIS KISUMU | Similarity: 0.80
OLKALAO NJAMBINI <--> OLKALAU NJABINI | Similarity: 0.90


EXCLUDE_WORDS = {''}

def should_exclude(cluster):
    return any(word in name for name in cluster for word in EXCLUDE_WORDS)

clusters = [c for c in clusters if not should_exclude(c)]

In [277]:
r_map={
    'KATITU KENDU BAY': 'KATITO KENDU BAY',
    'KATITO KENDU': 'KATITO KENDU BAY',
    'KILIFI MSA':'KILIFI MOMBASA',
    'THIUKA SUPER HIGHWAY':'THIKA SUPERHIGHWAY',
    'OLKALAO NJAMBINI':'OLKALAU NJABINI',
    'KENDU BAY KATITU':'KATITO KENDU BAY',
    'KENDU BAY KATITO':'KATITO KENDU BAY',
    'THIKA': 'THIKA SUPERHIGHWAY',
    'NAIVASHA NAIROBI HIGHWAY': 'NAIVASHA NAIROBI',
    'THIKA SUPER HIGHWAY':'THIKA SUPERHIGHWAY',
    'THIKA NAIROBI':'THIKA SUPERHIGHWAY',
    'MOMBASA':'NAIROBI MOMBASA',
    'ROAD MOMBASA':'NAIROBI MOMBASA',
    'SELASIE HAILE': 'HAILE SELASIE',
    'HAILE SELASIE AVENUE': 'HAILE SELASIE',
    'WAY WAIYAKI':'WAIYAKI',
    'MSA VOI':'MOMBASA VOI',
    'ROAD JOGOO':'JOGOO',
    

}
df2016['NORMALIZED_ROAD'] = df2016['NORMALIZED_ROAD'].apply(lambda x: r_map[x] if x in r_map else x)
df2016['NORMALIZED_ROAD'].value_counts()

NORMALIZED_ROAD
NAIROBI MOMBASA       40
THIKA SUPERHIGHWAY    15
NAKURU NAIROBI        11
NAKURU ELDORET        10
KITALE ELDORET         7
                      ..
KAPLONG BOMET          1
ROAD KANGUNDO          1
NDUMBERI KANNGA        1
HESHIMA                1
SPINE                  1
Name: count, Length: 203, dtype: int64

In [278]:
df2017.head()

,TIME 24 HOURS,BASE/SUB BASE,COUNTY,ROAD,PLACE,MV INVOLVED,BRIEF ACCIDENT DETAILS,NAME OF VICTIM,GENDER,AGE,CAUSE CODE,VICTIM,NO.,Date DD/MM/YYYY,Unnamed: 14
0,745,KISUMU,KISUMU,NAIROBI-BUSIA,KASAGAM FLY OVER,KBS 163T/ZD 9531 AXIOR TRAILER,THE VEHICLE LOST CONTROL AND HIT BARRIER LANDI...,UNKNOWN,F,A,29.0,PASSENGER,1.0,2017-11-06,2017-06-11
1,1430,NAROK,NAROK,NAROK-MAU,NAROK TOWN,KBS 518A HONDA CRV,THE VEHICLE KNOCKED DOWN THE VICTIM,UNKNOWN,M,80,68.0,PEDESTRIAN,1.0,2017-11-06,NaT
2,1515,LONDIANI,KERICHO,KERICHO-NAKURU,JUBERT AREA,KTCB 472K JOHN DEER,"THE VEHICLE LOST CONTROL,VEERING OFF THE ROAD ...",UNKNOWN,M,25,26.0,DRIVER,1.0,2017-11-06,NaT
3,1600,KIMILILI,BUNGOMA,BOKOLI-KIMILILI,BITUYU AREA,KBW 374M TOYOTA TOWNACE,THE VEHICLE KNOCKED DOWN THE VICTIM,UNKNOWN,M,31,7.0,PEDESTRIAN,1.0,2017-11-06,NaT
4,1747,NAKURU,NAKURU,PIPELINE ELEMENTAITA,PIPELINE AREA,KMDQ 433R DAYAN & KAR 843W,THE VEHICLE HIT THE MOTOR CYCLE,UNKNOWN,M,23,29.0,M/CYCLIST,1.0,2017-11-06,NaT


In [279]:
df2017['ROAD'].value_counts().head(20)

ROAD
NAIROBI-MOMBASA         15
WAIYAKI WAY             14
MOMBASA NAIROBI         12
MOMBASA ROAD            12
NAIROBI MOMBASA          9
THIKA SUPERHIGHWAY       8
THIKA SUPER HIGHWAY      6
NAIROBI NAIVASHA         5
KISUMU BUSIA             5
KANGUNDO ROAD            5
MOMBASA-NAIROBI          5
ELDORET NAKURU           5
NAIROBI MOMBASA ROAD     5
NAKURU-ELDORET           4
OUTERING ROAD            4
JOGOO ROAD               4
MERU MAUA                4
MAGADI ROAD              4
EASTERN BY PASS          4
NAIROBI-NAKURU           3
Name: count, dtype: int64

In [280]:
df2017['NORMALIZED_ROAD'] = df2017['ROAD'].apply(lambda x: normalize_road(x) if isinstance(x, str) else x)
df2017['NORMALIZED_ROAD'].value_counts()

NORMALIZED_ROAD
NAIROBI MOMBASA         52
NAKURU ELDORET          16
WAY WAIYAKI             14
NAIVASHA NAIROBI        13
ROAD MOMBASA            12
                        ..
PIPELINE ELEMENTAITA     1
KIMILILI BOKOLI          1
NAROK MAU                1
NAIROBI BUSIA            1
NAIROBI KANGUNDO         1
Name: count, Length: 425, dtype: int64

In [281]:
roads = df2017['NORMALIZED_ROAD'].unique().astype(str)

threshold = 0.8  # similarity from 0 to 1

similar_pairs = []

for i, road1 in enumerate(roads):
    for road2 in roads[i+1:]:
        ratio = difflib.SequenceMatcher(None, road1, road2).ratio()
        if ratio >= threshold:
            similar_pairs.append((road1, road2, ratio))

for r1, r2, score in similar_pairs:
    print(f"{r1} <--> {r2} | Similarity: {score:.2f}")

NAIROBI BUSIA <--> NAIROBI MSA | Similarity: 0.83
LONDIANI KERICHO <--> LONDAINI KERICHO | Similarity: 0.94
ROAD MOMBASA <--> RD MOMBASA | Similarity: 0.91
MARANI KISII <--> MARAMI KISII | Similarity: 0.92
NAIROBI MOMBASA <--> NAIROBIU MOMBASA | Similarity: 0.97
NAIROBI MOMBASA <--> NAIRBI MOMBASA | Similarity: 0.97
NAIROBI MOMBASA <--> NAIROBI MSA | Similarity: 0.85
NAROK MAIMAHIU <--> NAROK MAHIMAHIU | Similarity: 0.97
NAROK MAIMAHIU <--> NAROK MAI MAHIU | Similarity: 0.97
NAKURU ELDORET <--> NAURU ELDORET | Similarity: 0.96
EMBU CHUKA <--> MERU CHUKA | Similarity: 0.80
MOMBASA MALINDI <--> OLD MOMBASA MALINDI | Similarity: 0.88
KITALE ELDORET <--> ITEN ELDORET | Similarity: 0.85
WAY WAIYAKI <--> WAY WAIYIAKI | Similarity: 0.96
WANGARI MATHAI <--> WANGARI MATHAAI | Similarity: 0.97
WANGARI MATHAI <--> PROF.WANGARI MATHAI | Similarity: 0.85
WANGARI MATHAAI <--> PROF.WANGARI MATHAI | Similarity: 0.82
THIKA SUPERHIGHWAY <--> THIKA SUPER HIGHWAY | Similarity: 0.97
THIKA SUPERHIGHWAY <-->

roads = df2017['NORMALIZED_ROAD'].unique()

THRESHOLD = 80  # 0–100

similar_pairs = []

for r1, r2 in combinations(roads, 2):
    score = fuzz.token_sort_ratio(r1, r2)
    if score >= THRESHOLD:
        similar_pairs.append((r1, r2, score))
        
clusters = defaultdict(set)

for r1, r2, _ in similar_pairs:
    clusters[r1].add(r1)
    clusters[r1].add(r2)

clusters = list(clusters.values())
print(clusters)


EXCLUDE_WORDS = {'ROAD', 'HIGHWAY', 'BYPASS'}

def should_exclude(cluster):
    return any(word in name for name in cluster for word in EXCLUDE_WORDS)

clusters = [c for c in clusters if not should_exclude(c)]

In [282]:
r_map={
    'NAIROBI MOMBASA':[
        'MOMBASA',
        'ROAD MOMBASA',
        'MOMBASA ROAD',
        'NAIRBI MOMBASA',
        'NAIROBI MSA',
        'NAIROBIU MOMBASA',
        'NRB MSA'
    ],
    'WANGARI MATHAI': [
        'WANGARI MAATHAI',
        'WANGARI MATHAAI',
        'PROF. WANGARI MATHAI',
        'PROF.WANGARI MATHAI'
    ],
    'THIKA SUPERHIGHWAY': [
        'THIKA',
        'THIKA NAIROBI',
        'THIKA SUPERHIGH WAY',
        'THIKA SUPER HIGHWAY'
        
    ],
    'EASTERN BY PASS': ['EASTERN BYPASS'],
    'NORTHERN BY PASS': ['NORTHEN BY PASS'],
    'ROAD MURANGA':[
        'RD MURANGA',
        "ROAD MURANG'A"
    ],
    'KENDU BAY HOMA BAY':['KENDU BAY/ HOMA BAY'],
    'NAROK MAIMAHIU':[
        'NAROK MAI MAHIU',
        'NAROK MAHI MAHIU',
        'NAROK MAHIMAHIU'
    ],
    'NAKURU ELDORET':['NAURU ELDORET'],
    'OLKALAU KARIAMU': ['OL KALOU KARIAMU', 'OLKALOU KARIAMU'],
    'OLKALAU NYAHURURU': ['OL KALOU NYAHURURU', 'OLKALOU NYAHURURU'],
    'MAIMAHIU NAIVASHA': ['MAAI MAHIU NAIVASHA', 'MAI MAHIU NAIVASHA'],
    'WAIYAKI': ['WAY WAIYAKI', 'WAY WAIYIAKI'],
    'ROAD OUTERING': ['ROAD OUTERNG', 'ROAD OUTER RING'],
    'KENDU BAY KATITO': ['KENDU BAY KATITU', 'KINDU BAY KATITU', 'KEMDUBAY KATITO'],
    'JOGOO': ['ROAD JOGOO', 'JOGOO ROAD'],
    'MUMIAS BUSIA': ['MIMIAS BUSIA'],
    'UHURU HIGHWAY': ['UHURU HWY', 'UHURU HIGH WAY'],
    'NYERI NYAHURURU': ['NYERI IHURURU'],
    'NYERI NANYUKI': ['NYERI NANUKI']

    
}
def normalize_road_again(road):
    road = road.strip().upper()
    for canonical, variants in r_map.items():
        if road == canonical or road in variants:
            return canonical
    return road

df2017['NORMALIZED_ROAD'] = df2017['NORMALIZED_ROAD'].apply(lambda x: normalize_road_again(x) if isinstance(x, str) else x)

In [283]:
roads = df2017['NORMALIZED_ROAD'].unique().astype(str)

threshold = 0.8  # similarity from 0 to 1

similar_pairs = []

for i, road1 in enumerate(roads):
    for road2 in roads[i+1:]:
        ratio = difflib.SequenceMatcher(None, road1, road2).ratio()
        if ratio >= threshold:
            similar_pairs.append((road1, road2, ratio))

for r1, r2, score in similar_pairs:
    print(f"{r1} <--> {r2} | Similarity: {score:.2f}")
    

LONDIANI KERICHO <--> LONDAINI KERICHO | Similarity: 0.94
MARANI KISII <--> MARAMI KISII | Similarity: 0.92
EMBU CHUKA <--> MERU CHUKA | Similarity: 0.80
MOMBASA MALINDI <--> OLD MOMBASA MALINDI | Similarity: 0.88
KITALE ELDORET <--> ITEN ELDORET | Similarity: 0.85
ROAD KANGUNDO <--> RD KANGUNDO | Similarity: 0.92
EASTERN BY PASS <--> SOUTHERN BY PASS | Similarity: 0.84
NORTHERN BY PASS <--> SOUTHERN BY PASS | Similarity: 0.88
SOTIK KEROKA <--> SOTIK KERICHO | Similarity: 0.80
NGONG KISERIAN <--> RONGAI KISERIAN | Similarity: 0.83
SIGALAGALA BUKURA <--> SIGALAGALA BUTERE | Similarity: 0.82
MAGADI KISERIAN <--> RONGAI KISERIAN | Similarity: 0.80
MUMIAS BUTULA <--> MUMIAS BUSIA | Similarity: 0.80
NYANGUSU KEROKA <--> NYAGUSU KEROKA | Similarity: 0.97
KYENI KARURUMO <--> KYENI KARURUMO MARRUM | Similarity: 0.80
LONDAINI KERICHO <--> LETAIN KERICHO | Similarity: 0.80
KIBUKU EMBU <--> KIBUGU EMBU | Similarity: 0.91
SAGANA MAKUTANO <--> SAGANA MAKUTANO KENOL | Similarity: 0.83
RUIRU KIAMBU <

In [284]:
df2017['NORMALIZED_ROAD'].value_counts()

NORMALIZED_ROAD
NAIROBI MOMBASA       71
THIKA SUPERHIGHWAY    21
NAKURU ELDORET        17
WAIYAKI               16
NAIVASHA NAIROBI      13
                      ..
SEGEREK KAPOROK        1
TAWA KALAWANI          1
MIGORI AWENDO          1
NDUNYU MUNYANGE        1
TOWNSHIP KISUMU        1
Name: count, Length: 391, dtype: int64

In [285]:
df2016['GENDER'].value_counts()

GENDER
M           321
F            51
M & F         1
2M & 2F       1
2M, F         1
2F & M        1
2M & 3F       1
4F & INF      1
Name: count, dtype: int64

In [286]:
df2017=df2017.drop(columns=['NAME OF VICTIM','Unnamed: 14','MV INVOLVED','ROAD','GENDER' ])
df2017.columns

Index(['TIME 24 HOURS', 'BASE/SUB BASE', 'COUNTY', 'PLACE',
       'BRIEF ACCIDENT DETAILS', 'AGE', 'CAUSE CODE', 'VICTIM', 'NO.',
       'Date DD/MM/YYYY', 'NORMALIZED_ROAD'],
      dtype='object')

In [287]:
df2016=df2016.drop(columns=['Unnamed: 12','ROAD','GENDER'])
df2016.columns

Index(['TIME 24 HOURS', 'BASE/SUB BASE', 'COUNTY', 'PLACE',
       'BRIEF ACCIDENT DETAILS', 'AGE', 'CAUSE CODE', 'VICTIM', 'NO.',
       'Date DD/MM/YYYY', 'NORMALIZED_ROAD'],
      dtype='object')

In [288]:
df2017['BRIEF ACCIDENT DETAILS']

0      THE VEHICLE LOST CONTROL AND HIT BARRIER LANDI...
1                    THE VEHICLE KNOCKED DOWN THE VICTIM
2      THE VEHICLE LOST CONTROL,VEERING OFF THE ROAD ...
3                    THE VEHICLE KNOCKED DOWN THE VICTIM
4                        THE VEHICLE HIT THE MOTOR CYCLE
                             ...                        
735                THE VEHICLE KNOCKED DOWN A PEDESTRIAN
736                 THE CYCLE HIT THE STATIONARY VEHICLE
737                THE VEHICLE KNOCKED DOWN A PEDESTIRAN
738        THE VEHICLE COLLIDED HEAD ON WITH THE M/CYCLE
739                THE VEHICLE KNOCKED DOWN A PEDESTRIAN
Name: BRIEF ACCIDENT DETAILS, Length: 740, dtype: object

In [289]:
df2016['CAUSE CODE'].value_counts() 

CAUSE CODE
98.0    82
26.0    60
10.0    33
7.0     20
8.0     20
29.0    20
63.0    19
60.0    16
68.0    12
14.0     5
79.0     5
11.0     5
61.0     5
16.0     4
22.0     4
36.0     4
19.0     3
58.0     3
39.0     3
50.0     3
54.0     3
92.0     3
25.0     2
64.0     2
17.0     2
20.0     2
70.0     2
38.0     2
71.0     2
59.0     2
52.0     2
69.0     2
28.0     2
80.0     2
30.0     1
57.0     1
44.0     1
78.0     1
6.0      1
81.0     1
4.0      1
35.0     1
42.0     1
66.0     1
21.0     1
15.0     1
86.0     1
47.0     1
1.0      1
9.0      1
Name: count, dtype: int64

In [290]:
df2016.groupby('BRIEF ACCIDENT DETAILS')['CAUSE CODE'].first()

BRIEF ACCIDENT DETAILS
HEAD ON COLLISION                                                    7.0
HEAD ON COLLISION WITH A PEDAL CYCLIST                               8.0
HEAD ON COLLISION WITH AN ONCOMING VEHICLE                           8.0
HEAD ON COLLISION WITH ANOTHER TRAILER                              68.0
HEAD ON COLLISION WITH M/CYCLE                                      10.0
                                                                    ... 
THE VICTIM WAS KNOCKED DOWN BY THE UNKNOWN M/V                      63.0
THE VICTIM WAS ON TOP OF A TREE WHEN HE FELL DOWN ONTO THE TRUCK    64.0
THE VICTIM WAS RAN OVER BY THE VEHICLE                              98.0
UNKNOWN M/V HIT UNKNOWN PEDESTRIAN                                  98.0
UNKNOWN M/V KNOCKED DOWN A PEDESTRIAN                                7.0
Name: CAUSE CODE, Length: 147, dtype: float64

In [291]:
groups = (
    df2016[['CAUSE CODE', 'BRIEF ACCIDENT DETAILS']]
    .drop_duplicates()
    .groupby('BRIEF ACCIDENT DETAILS')['CAUSE CODE']
    .apply(list)
    .reset_index()
)
groups[groups['CAUSE CODE'].str.len() > 1]

,BRIEF ACCIDENT DETAILS,CAUSE CODE
0,HEAD ON COLLISION,"[7.0, 25.0, 10.0, 26.0, 8.0, 98.0, 22.0, 92.0,..."
2,HEAD ON COLLISION WITH AN ONCOMING VEHICLE,"[8.0, 1.0, 26.0]"
8,HIT AND RUN,"[98.0, 63.0, 7.0, 60.0, 61.0, 14.0, 86.0, 8.0,..."
19,THE CYCLE KNOCKED DOWN THE VICTIM,"[7.0, 29.0]"
21,THE CYCLE LOST CONTROL AND FELL DOWN,"[54.0, 36.0, 10.0]"
26,THE CYCLE LOST CONTROL AND LANDED INTO A DITCH,"[29.0, 98.0]"
37,THE CYCLE WAS HIT FROM THE REAR BY THE VEHICLE,"[19.0, 7.0]"
67,THE SAID M/V KNOCKED DOWN A PEDESTIAN,"[36.0, 63.0]"
68,THE SAID M/V KNOCKED DOWN PEDESTRIAN,"[60.0, 6.0]"
89,THE VEHICLE HIT THE M/CYCLE,"[7.0, 10.0, 26.0, 8.0]"


In [292]:
df2017.groupby('BRIEF ACCIDENT DETAILS')['CAUSE CODE'].first()

BRIEF ACCIDENT DETAILS
BUS DRIVER HIT A M/CYCLE CARRYING ONE F/A P/PASSENGER WHO DIED ON THE SPOT.                                                              10.0
BUS DRIVER KNOCKED DOWN M/A PEDESTRIAN SUSTAINED SERIOUS INJURIES AND DIED IN HOSPITAL                                                   60.0
COLLIDED HEAD ON                                                                                                                          8.0
DRIVER OF THE SAID VEHICLE RAN OVER THE SAID M/CYCLE FATALLY INJURING THE RIDER.                                                         10.0
DRIVER WAS OVERTAKING CARELESSLY AND IN THE PROCCESS COLLUDED HEAD ON WITH THE SAID M/BENZ.DRIVER OF TOYOTA COROLLA DIED ON THE SPOT.    10.0
                                                                                                                                         ... 
UNKNOWN M/V HIT ONE PERSON CROSSING THE ROAD KILLING HIM ON THE SPOT.                                                        

In [293]:
dffull=pd.concat([df2016,df2017],ignore_index=True)
dffull.shape

(1118, 11)

In [ ]:
v_map={
    'VEHICLE':['M/V','MOTOR VEHICLE','CAR','N/V','T/L/CRUISER','M/VEHICLE'],
    'CAR':['PRADO','PROBOX','TOYOTA ALLION','SALOON CAR','TOYOTA ISIS','TOYOTA FIELDER','TOYOTA MARK II','NISSAN BLUE','TOYOTA PROBOX','T/WISH','STARTLET','COLT','MITSUBISHI OUTLANDER','COROLLA','TOYOTA WISH','CRUISER','M/BENZ.DRIVER'],
    'TRUCK':['ISUZU','CANTER','LORRY','TANKER'],
    'MOTORCYCLE':['M/CYCLE','M/CYCE','M/CYCLIST','MOTORBIKE','MOTORCYCLE','RIDDER','M.CYCLIST', 'MOTOR CYCLE'],
    'BICYCLE':['PEDAL CYCLE','P/CYCLE','TRICYCLE','PEDAL CYCLIST','P/CYCLIST','CYCLIST','CYCLE'],
    'PICKUP':['P/UP',],
    'PEDESTRIAN':['J/PEDESTRIAN'],
    'TRAILER':['TRAILERS'],
    'MATATU':['T/MATATU']
}

def normalize_vehicles(text):
    for canonical, variants in v_map.items():
        for v in variants:
            text = text.replace(v, canonical)
    return text
dffull['TEXT_CLEAN'] = (
    dffull['BRIEF ACCIDENT DETAILS']
    .astype(str)
    .str.upper()
    .pipe(normalize_vehicles)
    .str.strip()
)



In [295]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1, 2),
    min_df=5
)
X = vectorizer.fit_transform(dffull['TEXT_CLEAN'])

In [302]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

scores = {}
for k in range(2, 50):
    km = KMeans(n_clusters=k, random_state=42)
    labels = km.fit_predict(X)
    scores[k] = silhouette_score(X, labels)

scores

{2: 0.18862088979343808,
 3: 0.2612371290174277,
 4: 0.2798372571510589,
 5: 0.4191229963183697,
 6: 0.41491272295206394,
 7: 0.43392971648596124,
 8: 0.4302434386515804,
 9: 0.4545287430653236,
 10: 0.463543976064179,
 11: 0.4717188770003679,
 12: 0.47967650561406927,
 13: 0.4873286447228015,
 14: 0.49533719558898126,
 15: 0.5011914662132638,
 16: 0.5068778294688585,
 17: 0.5105844209162408,
 18: 0.5094714540948542,
 19: 0.5139636781027513,
 20: 0.5190584477866949,
 21: 0.520775411980578,
 22: 0.5251336686313254,
 23: 0.5286055072322707,
 24: 0.5353399561679203,
 25: 0.5286638363007296,
 26: 0.5292845379584659,
 27: 0.5349682195329243,
 28: 0.5358866946301782,
 29: 0.5387456433364751,
 30: 0.5428666937719528,
 31: 0.5486457943919887,
 32: 0.5469055963483487,
 33: 0.5495682385692614,
 34: 0.5601844764809285,
 35: 0.561220764357871,
 36: 0.5611096209397873,
 37: 0.5634203969992172,
 38: 0.5642203955962659,
 39: 0.5659823769567227,
 40: 0.5673819178020054,
 41: 0.5680485716258822,
 42: 0

In [300]:
kmeans=KMeans(n_clusters=15, random_state=42)
dffull['CLUSTER']=kmeans.fit_predict(X)

In [301]:
import numpy as np

terms = vectorizer.get_feature_names_out()

for i in range(15):
    print(f"\nCluster {i}")
    center = kmeans.cluster_centers_[i]
    top_terms = np.argsort(center)[-10:]
    print([terms[j] for j in top_terms])


Cluster 0
['cycle hit', 'unknown', 'lorry', 'vehicle', 'vehicle hit', 'hit', 'cycle', 'hit motor', 'motor cycle', 'motor']

Cluster 1
['vehicles', 'vehicle', 'landed', 'ditch', 'landed ditch', 'victim', 'knocked victim', 'cycle', 'knocked', 'cycle knocked']

Cluster 2
['vehicle hit', 'vehicle knocked', 'vehicle lost', 'vehicle motion', 'vehicle rammed', 'victim ran', 'victim fell', 'hit', 'run', 'hit run']

Cluster 3
['vehicles', 'vehicle motion', 'vehicle lost', 'vehicle rammed', 'cycle', 'trailer', 'vehicle', 'head', 'head collision', 'collision']

Cluster 4
['road knocked', 'vehicle', 'control', 'lost control', 'lost', 'vehicle lost', 'road', 'control veered', 'veered road', 'veered']

Cluster 5
['rolled times', 'times', 'cycle', 'control hit', 'control rolled', 'cycle lost', 'rolled', 'lost control', 'control', 'lost']

Cluster 6
['lorry', 'matatu', 'vehicle', 'trailer', 'vehicle rammed', 'cycle', 'cycle rammed', 'rear', 'rammed rear', 'rammed']

Cluster 7
['vehicles', 'vehicle ra